In [1]:
import os
from os.path import isfile
import sys
import glob

os.environ["CUDA_VISIBLE_DEVICES"]="4"
#os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L1, L2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler

import mlflow
import mlflow.tensorflow

2024-09-29 19:48:41.401198: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-29 19:48:41.533444: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
mlflow.tensorflow.autolog()
mlflow.set_experiment("HSEL+CPL q")

2024/09/29 19:48:45 INFO mlflow.tracking.fluent: Experiment with name 'HSEL+CPL q' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///data/nature_run/work/src/mlruns/424939423499244567', creation_time=1727653725883, experiment_id='424939423499244567', last_update_time=1727653725883, lifecycle_stage='active', name='HSEL+CPL q', tags={}>

In [3]:
def get_data(filelist):
    hsel_val = []
    scalar_val = []
    table_val = []
    cpl_val = []

    for i in filelist:
        # Keys: 'mh', 'hsel', 'scalar', 'table']
        tmp = np.load(i)
        hsel_val.append(tmp["hsel"])
        scalar_val.append(tmp["scalar"])
        table_val.append(tmp["table"])
        cpl_val.append(tmp["cpl"])

    return np.vstack(hsel_val), np.vstack(cpl_val), np.vstack(scalar_val), np.vstack(table_val)


train = ['/data/nature_run/fulldays_reduced/all_cpl_20060815.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20060915.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20061015.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20060515.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20060715.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20061115.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20060315.npz', 
         '/data/nature_run/fulldays_reduced/all_cpl_20061215.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20060615.npz']

test = ['/data/nature_run/fulldays_reduced/all_cpl_20060803.npz']

validation = ['/data/nature_run/fulldays_reduced/all_cpl_20060803.npz']

hsel_train, cpl_train, scalar_train, table_train = get_data(train)
hsel_test, cpl_test, scalar_test, table_test = get_data(test)
hsel_val, cpl_val, scalar_val, table_val = get_data(validation)



In [ ]:
#hsel_train, table_train, scalar_train, hsel_test, table_test, scalar_test = dataload("mh")

In [ ]:
#hsel_train, table_train, scalar_train, hsel_test, table_test, scalar_test = dataload("mh")
#hsel_val = hsel_test.copy()
#table_val = table_test.copy()
#scalar_val = scalar_test.copy()


In [ ]:
print(hsel_train.shape)
print(cpl_train.shape)
print(table_train.shape)
print(scalar_train.shape)

In [4]:
# What should we do
# 1) Only use train scalars
# 2) Scale seperateley


# Presssure ####
spress_test = scalar_test[:, 3].reshape(-1, 1)
spress_train = scalar_train[:, 3].reshape(-1, 1)
spress_val = scalar_val[:, 3].reshape(-1, 1)

mins = np.min(spress_train, axis=0)
maxs = np.max(spress_train, axis=0)
np.savez("spress_scalar_cpl.npz", mins=mins, maxs=maxs)

spress_test = (spress_test - mins)/(maxs - mins)
spress_train = (spress_train - mins)/(maxs - mins)
spress_val = (spress_val - mins)/(maxs - mins)


# Labels ####
q_train = table_train[:, :, 2]
q_test = table_test[:, :, 2]
q_val = table_val[:, :, 2]

# Spectra ####
mins = np.min(hsel_train, axis=0)
maxs = np.max(hsel_train, axis=0)

np.savez("minimac_scaling_factors_cpl_mh.npz", maxs=maxs, mins=mins)

hsel_train = np.nan_to_num(hsel_train)
hsel_test = np.nan_to_num(hsel_test)
hsel_val = np.nan_to_num(hsel_val)

hsel_train = (hsel_train - mins)/(maxs - mins)
hsel_test = (hsel_test - mins)/(maxs - mins)
hsel_val = (hsel_val - mins)/(maxs - mins)

# CPL #####
mins = np.min(cpl_train, axis=0)
maxs = np.max(cpl_train, axis=0)
np.savez("minimac_scaling_factors_cpl_cpl.npz", maxs=maxs, mins=mins)

cpl_train = (cpl_train - mins)/(maxs - mins)
cpl_test = (cpl_test - mins)/(maxs - mins)
cpl_val = (cpl_val - mins)/(maxs - mins)

with tf.device('/cpu:0'):
    # train
    x_train = {'rad': tf.convert_to_tensor(hsel_train, np.float32),
               'spress': tf.convert_to_tensor(spress_train, np.float32),
               'cpl': tf.convert_to_tensor(cpl_train, np.float32)}
    del hsel_train
    del spress_train
    
    y_train =  tf.convert_to_tensor(q_train, np.float32)
    del q_train

    
    # Val
    x_val = {'rad': tf.convert_to_tensor(hsel_val, np.float32),
             'spress': tf.convert_to_tensor(spress_val, np.float32),
             'cpl': tf.convert_to_tensor(cpl_val, np.float32)}
    del hsel_val
    del spress_val
    
    y_val =  tf.convert_to_tensor(q_val, np.float32)
    del q_val

    
    # Test
    x_test = {'rad': tf.convert_to_tensor(hsel_test, np.float32),
              'spress': tf.convert_to_tensor(spress_test, np.float32),
              'cpl': tf.convert_to_tensor(cpl_test, np.float32)}
    del hsel_test
    del spress_test
    
    y_test =  tf.convert_to_tensor(q_test, np.float32)
    del q_test


2024-09-29 21:00:30.146843: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-29 21:00:30.722372: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:222] Using CUDA malloc Async allocator for GPU: 0
2024-09-29 21:00:30.723385: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31017 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:88:00.0, compute capability: 7.0


In [ ]:
# print(x_test)
# print(x_val)
# print(y_train.shape)
#a = np.argwhere(np.isnan(x_train['rad']))
#print(np.sum(np.isnan(x_train['rad'])))
#np.nan_to_num(x_train['rad'], copy=False)
#print(np.sum(np.isnan(x_train['rad'])))
#print(a)
#print(len(a))

np.sum(np.isnan(x_train['rad']))

In [5]:
def build_ae_model(config):
    # HSEL 22
    mh = Input(shape=(config["shape"],), name="rad")
    spress = Input(shape=(1,), name="spress")
    #cpl = Input(shape=(733,), name="cpl")
    # [ha, hb, hc, hd, hw, mh]

    encoder = load_model("encoder_3_mae.keras")
    encoder.trainable = False

    #cpl_reduced = Dense(128, activation=config["activation"])(cpl)
    #cpl_reduced = Dense(32, activation=config["activation"])(cpl_reduced)
    
    enc = encoder(mh)
    x = Concatenate()([enc, spress])
    for i in range(config["num_layers"]):
        x =  Dropout(config["dropout"])(Dense(config["num_neurons"], 
                                             activation=config["activation"])(x))
    outputs = Dense(config['output'], name="Temp")(x)

    model = Model(inputs=[mh, spress], outputs=outputs, name="q")
    model.compile(optimizer="adam", loss='mae')

    return model

config = {'shape': 1957,
          'output': 72,
          'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
          'num_layers': np.random.choice([3, 4, 5, 10]),
          'num_neurons': np.random.choice([16, 32, 64, 128]),
          'activation': np.random.choice(['gelu', 'gelu', 'relu'])}

model = build_ae_model(config)
model.summary()


Model: "q"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 rad (InputLayer)               [(None, 1957)]       0           []                               
                                                                                                  
 model_1 (Functional)           (None, 64)           275468      ['rad[0][0]']                    
                                                                                                  
 spress (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 65)           0           ['model_1[0][0]',                
                                                                  'spress[0][0]']                 

In [6]:
def build_ae_cpl_model(config):
    # ATMS 22
    mh = Input(shape=(config["shape"],), name="rad")
    spress = Input(shape=(1,), name="spress")
    cpl = Input(shape=(733,), name="cpl")
    # [ha, hb, hc, hd, hw, mh]

    encoder = load_model("encoder_3_mae.keras")
    encoder.trainable = False

    cpl_reduced = Dense(128, activation=config["activation"])(cpl)
    cpl_reduced = Dense(32, activation=config["activation"])(cpl_reduced)
    
    enc = encoder(mh)
    x = Concatenate()([enc, spress, cpl_reduced])
    for i in range(config["num_layers"]):
        x =  Dropout(config["dropout"])(Dense(config["num_neurons"], 
                                             activation=config["activation"])(x))
    outputs = Dense(config['output'], name="Temp")(x)

    model = Model(inputs=[mh, spress, cpl], outputs=outputs, name="q")
    model.compile(optimizer="adam", loss='mae')

    return model

config = {'shape': 1957,
          'output': 72,
          'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
          'num_layers': np.random.choice([3, 4, 5, 10]),
          'num_neurons': np.random.choice([16, 32, 64, 128]),
          'activation': np.random.choice(['gelu', 'gelu', 'relu'])}

model = build_ae_model(config)
model.summary()


Model: "q"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 rad (InputLayer)               [(None, 1957)]       0           []                               
                                                                                                  
 model_1 (Functional)           (None, 64)           275468      ['rad[0][0]']                    
                                                                                                  
 spress (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  
 concatenate_1 (Concatenate)    (None, 65)           0           ['model_1[0][0]',                
                                                                  'spress[0][0]']                 

In [9]:
print("DING")

DING


In [ ]:
# AE

for _ in range(100):
    config = {'shape': 1957,
              'output': 72,
              'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
              'num_layers': np.random.choice([3, 4, 5, 10]),
              'num_neurons': np.random.choice([16, 32, 64, 128]),
              'activation': np.random.choice(['gelu', 'gelu', 'relu'])}

    model = build_ae_cpl_model(config)
    
    with mlflow.start_run() as run:
        model_checkpoint_callback = ModelCheckpoint(
            filepath=f"models/model_{run.info.run_id}.keras",
            save_weights_only=False,
            monitor='val_loss',
            mode='min',
            save_best_only=True)
        callback = EarlyStopping(monitor='val_loss', patience=8, verbose=0)
        mlflow.log_param("Autoencoder", True)
        mlflow.log_param("CPL", True)

        history = model.fit(x_train, y_train, epochs=1000, batch_size=2000,
                            validation_data=(x_val, y_val),
                            verbose=2,
                            callbacks=[callback, model_checkpoint_callback])
        
        evaled_test = model.evaluate(x_train, y_train, batch_size=100, verbose=0)
        evaled_train = model.evaluate(x_test, y_test, batch_size=100, verbose=0)

        mlflow.log_metric("test_loss", evaled_test)
        mlflow.log_metric("train_loss", evaled_train)

        plt.plot(history.history["loss"])
        plt.plot(history.history["val_loss"])
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plotname = f"plots/loss_plot_{run.info.run_id}.png"
        plt.savefig(plotname)
        plt.close()

        # Log the plot as an artifact
        mlflow.log_artifact(plotname)
        #model_path = "modles"
        #mlflow.tensorflow.log_model(tf_saved_model_dir=model_path, artifact_path="model")


2024/09/29 21:20:17 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/29 21:20:17 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


Epoch 1/1000
4208/4208 - 71s - loss: 0.0014 - val_loss: 5.5836e-04 - 71s/epoch - 17ms/step
Epoch 2/1000
4208/4208 - 69s - loss: 5.8657e-04 - val_loss: 5.4408e-04 - 69s/epoch - 16ms/step
Epoch 3/1000
4208/4208 - 67s - loss: 5.6945e-04 - val_loss: 5.1746e-04 - 67s/epoch - 16ms/step
Epoch 4/1000
4208/4208 - 67s - loss: 5.1922e-04 - val_loss: 4.5836e-04 - 67s/epoch - 16ms/step
Epoch 5/1000
4208/4208 - 66s - loss: 5.0480e-04 - val_loss: 4.4960e-04 - 66s/epoch - 16ms/step
Epoch 6/1000
4208/4208 - 65s - loss: 5.0404e-04 - val_loss: 4.4677e-04 - 65s/epoch - 16ms/step
Epoch 7/1000
4208/4208 - 66s - loss: 5.0209e-04 - val_loss: 4.3893e-04 - 66s/epoch - 16ms/step
Epoch 8/1000
4208/4208 - 65s - loss: 5.0082e-04 - val_loss: 4.5013e-04 - 65s/epoch - 16ms/step
Epoch 9/1000
4208/4208 - 64s - loss: 5.0015e-04 - val_loss: 4.4245e-04 - 64s/epoch - 15ms/step
Epoch 10/1000
4208/4208 - 65s - loss: 5.0024e-04 - val_loss: 4.5449e-04 - 65s/epoch - 15ms/step
Epoch 11/1000
4208/4208 - 65s - loss: 4.9890e-04 - va

2024/09/29 21:37:55 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/data/jmackin1/miniconda/envs/tf-gpu/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


2024/09/29 21:42:54 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/29 21:42:54 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


Epoch 1/1000
4208/4208 - 64s - loss: 0.0019 - val_loss: 6.2061e-04 - 64s/epoch - 15ms/step
Epoch 2/1000
4208/4208 - 65s - loss: 6.3472e-04 - val_loss: 5.5750e-04 - 65s/epoch - 15ms/step
Epoch 3/1000
4208/4208 - 64s - loss: 6.0864e-04 - val_loss: 5.2478e-04 - 64s/epoch - 15ms/step
Epoch 4/1000
4208/4208 - 62s - loss: 5.7556e-04 - val_loss: 4.8087e-04 - 62s/epoch - 15ms/step
Epoch 5/1000
4208/4208 - 61s - loss: 5.5980e-04 - val_loss: 4.8991e-04 - 61s/epoch - 15ms/step
Epoch 6/1000
4208/4208 - 62s - loss: 5.5705e-04 - val_loss: 4.8246e-04 - 62s/epoch - 15ms/step
Epoch 7/1000
4208/4208 - 62s - loss: 5.5264e-04 - val_loss: 4.7132e-04 - 62s/epoch - 15ms/step
Epoch 8/1000
4208/4208 - 62s - loss: 5.5323e-04 - val_loss: 4.7163e-04 - 62s/epoch - 15ms/step
Epoch 9/1000
4208/4208 - 61s - loss: 5.5331e-04 - val_loss: 4.8736e-04 - 61s/epoch - 15ms/step
Epoch 10/1000
4208/4208 - 61s - loss: 5.5225e-04 - val_loss: 4.8254e-04 - 61s/epoch - 15ms/step
Epoch 11/1000
4208/4208 - 62s - loss: 5.5198e-04 - va

...

Epoch 1/1000
4208/4208 - 63s - loss: 0.0015 - val_loss: 4.6636e-04 - 63s/epoch - 15ms/step
Epoch 2/1000
4208/4208 - 62s - loss: 4.9999e-04 - val_loss: 4.4467e-04 - 62s/epoch - 15ms/step
Epoch 3/1000
4208/4208 - 61s - loss: 4.8272e-04 - val_loss: 4.4052e-04 - 61s/epoch - 14ms/step
Epoch 4/1000
4208/4208 - 62s - loss: 4.6855e-04 - val_loss: 4.2876e-04 - 62s/epoch - 15ms/step
Epoch 5/1000
4208/4208 - 61s - loss: 4.6278e-04 - val_loss: 4.2939e-04 - 61s/epoch - 15ms/step
Epoch 6/1000
4208/4208 - 62s - loss: 4.6107e-04 - val_loss: 4.2478e-04 - 62s/epoch - 15ms/step
Epoch 7/1000
4208/4208 - 64s - loss: 4.6006e-04 - val_loss: 4.1804e-04 - 64s/epoch - 15ms/step
Epoch 8/1000
4208/4208 - 62s - loss: 4.5876e-04 - val_loss: 4.1937e-04 - 62s/epoch - 15ms/step
Epoch 9/1000
4208/4208 - 60s - loss: 4.5787e-04 - val_loss: 4.2690e-04 - 60s/epoch - 14ms/step
Epoch 10/1000
4208/4208 - 65s - loss: 4.5146e-04 - val_loss: 3.9778e-04 - 65s/epoch - 15ms/step
Epoch 11/1000
4208/4208 - 64s - loss: 4.4892e-04 - va

2024/09/30 06:19:46 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/30 06:19:46 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


Epoch 1/1000
4208/4208 - 59s - loss: 7.6192e-04 - val_loss: 3.3932e-04 - 59s/epoch - 14ms/step
Epoch 2/1000
4208/4208 - 58s - loss: 3.0935e-04 - val_loss: 3.0657e-04 - 58s/epoch - 14ms/step
Epoch 3/1000
4208/4208 - 60s - loss: 2.8418e-04 - val_loss: 2.8129e-04 - 60s/epoch - 14ms/step
Epoch 4/1000
4208/4208 - 61s - loss: 2.7100e-04 - val_loss: 2.8160e-04 - 61s/epoch - 15ms/step
Epoch 5/1000
4208/4208 - 56s - loss: 2.6291e-04 - val_loss: 2.8469e-04 - 56s/epoch - 13ms/step
Epoch 6/1000
4208/4208 - 57s - loss: 2.5670e-04 - val_loss: 2.8424e-04 - 57s/epoch - 13ms/step
Epoch 7/1000
4208/4208 - 57s - loss: 2.5283e-04 - val_loss: 2.8308e-04 - 57s/epoch - 14ms/step
Epoch 8/1000
4208/4208 - 57s - loss: 2.4948e-04 - val_loss: 2.7214e-04 - 57s/epoch - 13ms/step
Epoch 9/1000
4208/4208 - 57s - loss: 2.4604e-04 - val_loss: 2.6624e-04 - 57s/epoch - 14ms/step
Epoch 10/1000
4208/4208 - 60s - loss: 2.4427e-04 - val_loss: 2.7876e-04 - 60s/epoch - 14ms/step
Epoch 11/1000
4208/4208 - 58s - loss: 2.4182e-04 

2024/09/30 06:41:55 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/30 06:41:55 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


Epoch 1/1000
4208/4208 - 55s - loss: 0.0016 - val_loss: 8.9645e-04 - 55s/epoch - 13ms/step
Epoch 2/1000
4208/4208 - 55s - loss: 8.6464e-04 - val_loss: 8.9357e-04 - 55s/epoch - 13ms/step
Epoch 3/1000
4208/4208 - 58s - loss: 8.6482e-04 - val_loss: 9.0264e-04 - 58s/epoch - 14ms/step
Epoch 4/1000
4208/4208 - 55s - loss: 8.6472e-04 - val_loss: 8.9551e-04 - 55s/epoch - 13ms/step
Epoch 5/1000
4208/4208 - 55s - loss: 8.6487e-04 - val_loss: 9.0427e-04 - 55s/epoch - 13ms/step
Epoch 6/1000
4208/4208 - 57s - loss: 8.6463e-04 - val_loss: 9.0580e-04 - 57s/epoch - 13ms/step
Epoch 7/1000
4208/4208 - 55s - loss: 8.6458e-04 - val_loss: 8.9706e-04 - 55s/epoch - 13ms/step
Epoch 8/1000
4208/4208 - 57s - loss: 8.6457e-04 - val_loss: 9.0270e-04 - 57s/epoch - 13ms/step
Epoch 9/1000
4208/4208 - 54s - loss: 8.6474e-04 - val_loss: 9.0267e-04 - 54s/epoch - 13ms/step
Epoch 10/1000
4208/4208 - 55s - loss: 8.6490e-04 - val_loss: 8.9597e-04 - 55s/epoch - 13ms/step
1/1 [==============================] - 0s 173ms/step


In [ ]:
# regs

for _ in range(100):
    config = {'shape': 1957,
              'output': 72,
              'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
              'num_layers': np.random.choice([3, 4, 5, 10]),
              'num_neurons': np.random.choice([16, 32, 64, 128]),
              'activation': np.random.choice(['gelu', 'gelu', 'relu'])}
    
    model = build_ae_model(config)
    
    with mlflow.start_run() as run:
        model_checkpoint_callback = ModelCheckpoint(
            filepath=f"models/model_{run.info.run_id}.keras",
            save_weights_only=False,
            monitor='val_loss',
            mode='min',
            save_best_only=True)
        callback = EarlyStopping(monitor='val_loss', patience=8, verbose=0)
        mlflow.log_param("Autoencoder", True)
        mlflow.log_param("CPL", False)

        history = model.fit(x_train, y_train, epochs=1000, batch_size=2000,
                            validation_data=(x_val, y_val),
                            verbose=2,
                            callbacks=[callback, model_checkpoint_callback])
        
        evaled_test = model.evaluate(x_train, y_train, batch_size=100, verbose=0)
        evaled_train = model.evaluate(x_test, y_test, batch_size=100, verbose=0)

        mlflow.log_metric("test_loss", evaled_test)
        mlflow.log_metric("train_loss", evaled_train)

        plt.plot(history.history["loss"])
        plt.plot(history.history["val_loss"])
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plotname = f"plots/loss_plot_{run.info.run_id}.png"
        plt.savefig(plotname)
        plt.close()

        # Log the plot as an artifact
        mlflow.log_artifact(plotname)
        #model_path = "modles"
        #mlflow.tensorflow.log_model(tf_saved_model_dir=model_path, artifact_path="model")
